추천 시스템 방식

- 콘텐츠 기반 필터링 (Content Based Filtering)

- 협업 필터링 (Collaborative Filtering)

- 위 두가지를 결합한 Hybrid 방식

* 콘텐츠 기반 필터링 실습 (Contents Based Filtering)

- 상품을 구성하는 다양한 콘텐츠들을 텍스트 기반 유사도(코사인 유사도)로 비교하여 추천

콘텐츠 기반 필터링 구현 프로세스

1. 콘텐츠에 대한 텍스트 정보들을 피쳐 벡터화
2. 코사인 유사도로 콘텐츠별 유사도 계산
3. 콘텐츠 별로 가중 평점을 계산
4. 유사도가 높은 콘텐츠 중 평점이 좋은 콘텐츠 순으로 추천

캐글 데이터 셋 사용

- https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [1]:
# 필요 라이브러리 import

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

# str 형태를 list 형태로 바꿔주는 라이브러리
from ast import literal_eval

In [2]:
movies = pd.read_csv('../src/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [3]:
# 리스트 형태로 보이는 값들이 object 타입인 것을 확인할 수 있고,
# 각 컬럼의 value 값들을 보면 str 타입인 것을 확인할 수 있음.
print(type(movies['genres'][0]))
movies[['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']].info()

<class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   genres                4803 non-null   object
 1   keywords              4803 non-null   object
 2   production_companies  4803 non-null   object
 3   production_countries  4803 non-null   object
 4   spoken_languages      4803 non-null   object
dtypes: object(5)
memory usage: 187.7+ KB


In [4]:
# list 타입으롭 변환하기
movies['genres'] = movies['genres'].apply(literal_eval)
movies['keywords'] = movies['keywords'].apply(literal_eval)
movies['production_companies'] = movies['production_companies'].apply(literal_eval)
movies['production_countries'] = movies['production_countries'].apply(literal_eval)
movies['spoken_languages'] = movies['spoken_languages'].apply(literal_eval)
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
# list로 변경됨을 확인할 수 있음
print(type(movies['genres'][0]))
movies['genres'][0]

<class 'list'>


[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [6]:
# 컬럼의 dict 안의 키워드들만 뽑아내기
movies['genres'] = movies['genres'].apply(lambda x: ' '.join([y['name'] for y in x]))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join([y['name'] for y in x]))
movies[['genres', 'keywords']]

,genres,keywords
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...
2,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...
...,...,...
4798,Action Crime Thriller,united states–mexico barrier legs arms paper k...
4799,Comedy Romance,
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...
4801,,


In [10]:
'''CountVector Test
    - CounterVector 수행 과정
    1. 문서를 토큰 리스트로 변환
    2. 각 문서에서 토큰의 출현 빈도를 셈
    3. 각 문서에서 BOW(Bag of Words) 인코딩 벡터로 변환
'''

from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'this is the second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?'
]

test_vect = CountVectorizer()
test_vect.fit(corpus)
print(test_vect.vocabulary_)
test_mat = test_vect.transform(corpus)
print(test_mat)
print(test_mat.shape)

{'this': 9, 'is': 3, 'the': 7, 'first': 2, 'document': 1, 'second': 6, 'and': 0, 'third': 8, 'one': 5, 'last': 4}
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 7)	1
  (0, 9)	1
  (1, 1)	1
  (1, 3)	1
  (1, 6)	1
  (1, 7)	1
  (1, 9)	1
  (2, 0)	1
  (2, 5)	1
  (2, 7)	1
  (2, 8)	1
  (3, 1)	1
  (3, 2)	1
  (3, 3)	1
  (3, 7)	1
  (3, 9)	1
  (4, 1)	1
  (4, 4)	1
  (4, 7)	1
(5, 10)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# 문서를 토큰 리스트로 변환
# 각 문서에서 토큰의 출현 빈도 계산
# 각 문서를 BOW(Bag of Words) 인코딩 벡터로 변환한다.

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
count_vect

CountVectorizer(min_df=0, ngram_range=(1, 2))

In [13]:
# 장르 메트릭스 4803행, 276열 생성
genre_mat = count_vect.fit_transform(movies['genres'])
genre_mat.shape

(4803, 276)

In [14]:
# 코사인 유사도에 의해 4803개 영화 각각 영화별 유사도 계산
# 코사인 유사도
'''Cosine similarity: 내적공간의 두 벡터간 각도의 코사인 값을 이용하여 측적된 벡터간의 유사한 정도
    각도가 0°일때의 코사인 값은 1이며, 다른 모든 각도의 코사인 값은 1보다 작다.
    따라서 이 값은 벡터의 크기가 아닌 방향의 유사도를 판단하는 목적으로 사용되며
    두 벡터의 방향이 완전히 같을 경우 1
    90°의 각을 이룰경우 0
    180°로 완전히 반대 방향인 경우 -1의 값을 갖는다.
    이 때, 벡터의 크기는 값에 아무런 영향을 미치지 않는다.
    
    코사인 유사도는 어떤 개수의 차원에도 적요이 가능한데 특히 다차원 양수 공간에서의 유사도 측정에 자주 이용된다.
    EX.) 정보 검색 및 텍스트 마이닝 분야에서, 단어 하나 하나는 각각의 차원을 구성하고 문서는 각 단어가 문서에서 나타나는 회수로 표현되는 벡터값을 가진다.
    이러한 다차원 공간에서 코사인 유사도는 두 문서의 유사를 측정하는 매우 유용한 방법이다.
    '''
from sklearn.metrics.pairwise import cosine_similarity
gerne_sim = cosine_similarity(genre_mat, genre_mat)
print(gerne_sim.shape)
print(gerne_sim[:5])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [16]:
gerne_sim.argsort()[:, ::-1][0]

array([   0, 3494,  813, ..., 3038, 3037, 2401], dtype=int64)

In [28]:
genre_sim_sorted_idx = gerne_sim.argsort()[:, ::-1]
genre_sim_sorted_idx

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

In [ ]:
# 추천 장르 코사인 유사도에 의해 영화를 추천하는 함수
def find_sim_mv_ver1(df:pd.DataFrame, sorted_idx, title: str, top_n=10):
    # 인자로 입력된 movie_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title]
    
    # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_idx 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_idx = title_movie.index.values
    similar_indexes = sorted_idx[title_idx, :top_n]

In [19]:
# 'title' 컬럼이 입력된 title_name 값인 DataFrame 추출
title_movie = movies[movies['title'] == 'The Godfather']
title_movie

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3337,6000000,Drama Crime,http://www.thegodfather.com/,238,italy love at first sight loss of father patri...,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",143.659698,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1972-03-14,245066411,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,8.4,5893


In [25]:
# title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
title_index = title_movie.index.values
title_index

array([3337], dtype=int64)

In [31]:
# sorted_idx 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
similar_indexes = genre_sim_sorted_idx[title_index, :10]
similar_indexes

array([[2731, 1243, 3636, 1946, 2640, 4065, 1847, 4217,  883, 3866]],
      dtype=int64)

In [33]:
similar_indexes = similar_indexes.reshape(-1)

In [35]:
similar_movies = movies.iloc[similar_indexes]
similar_movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2731,13000000,Drama Crime,NaN,240,italo-american cuba vororte melancholy praise ...,en,The Godfather: Part II,In the continuing saga of the Corleone crime f...,105.792936,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1974-12-20,47542841,200.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"I don't feel I have to wipe everybody out, Tom...",The Godfather: Part II,8.3,3338
1243,500000,Drama Crime,NaN,203,epilepsy protection money secret love money re...,en,Mean Streets,"A small-time hood must choose from among love,...",17.002096,"[{'name': 'Scorsese Productions', 'id': 120}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1973-10-02,3000000,110.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,You don't make up for your sins in church. You...,Mean Streets,7.2,345
3636,5000000,Drama Crime,NaN,36351,suicide drug dealer redemption addict existent...,en,Light Sleeper,A drug dealer with upscale clientele is having...,6.063868,"[{'name': 'Fine Line Features', 'id': 8}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1992-08-21,0,103.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He was a good man in a deadly business. She wa...,Light Sleeper,5.7,15
1946,25000000,Drama Crime,NaN,11699,police brutality organized crime policeman ill...,en,The Bad Lieutenant: Port of Call - New Orleans,"Terrence McDonagh, a New Orleans Police sergea...",17.339852,"[{'name': 'Saturn Films', 'id': 831}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-09-11,10589102,122.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,The only criminal he can't catch is himself.,The Bad Lieutenant: Port of Call - New Orleans,6.0,326
2640,8000000,Drama Crime,NaN,400,father son relationship bounty hunter boat way...,en,Things to Do in Denver When You're Dead,A mafia film in Tarantino style with a star-st...,6.932221,"[{'name': 'Miramax Films', 'id': 14}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-01,529766,116.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Protect. Love. Honor. Avenge.,Things to Do in Denver When You're Dead,6.7,85
4065,2100000,Drama Crime,http://miamericamovie.net/,364083,new york state hate crime,en,Mi America,A hate-crime has been committed in a the small...,0.039007,"[{'name': 'Industrial House Films', 'id': 65589}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-10-16,3330,126.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,NaN,Mi America,0.0,0
1847,25000000,Drama Crime,http://www.warnerbros.com/goodfellas,769,prison based on novel florida 1970s mass murde...,en,GoodFellas,"The true story of Henry Hill, a half-Irish, ha...",63.654244,"[{'name': 'Winkler Films', 'id': 8880}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-09-12,46836394,145.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,Three Decades of Life in the Mafia.,GoodFellas,8.2,3128
4217,1500000,Drama Crime,NaN,9344,puberty first time,en,Kids,A controversial portrayal of teens in New York...,13.291991,"[{'name': 'Miramax Films', 'id': 14}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-07-28,7412216,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Kids,6.8,279
883,52000000,Drama Crime,https://www.facebook.com/CatchMeIfYouCanMovie,640,con man biography fbi agent overhead camera sh...,en,Catch Me If You Can,"A true story about Frank Abagnale Jr. who, bef...",73.944049,"[{'name': 'Kemp Company', 'id': 367}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",2002-12-25,352114312,141.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The true story of a real fake.,Catch Me If You Can,7.7,3795
3866,3300000,Drama Crime,

In [37]:
similar_movies[['title', 'vote_average', 'genres', 'vote_count']]

,title,vote_average,genres,vote_count
2731,The Godfather: Part II,8.3,Drama Crime,3338
1243,Mean Streets,7.2,Drama Crime,345
3636,Light Sleeper,5.7,Drama Crime,15
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,Drama Crime,326
2640,Things to Do in Denver When You're Dead,6.7,Drama Crime,85
4065,Mi America,0.0,Drama Crime,0
1847,GoodFellas,8.2,Drama Crime,3128
4217,Kids,6.8,Drama Crime,279
883,Catch Me If You Can,7.7,Drama Crime,3795
3866,City of God,8.1,Drama Crime,1814


#### 위에서 평가 횟수가 현저히 적은 영화들이 추천되는 것도 있음.
#### 평가 횟수를 반영한 추천 시스템이 필요함.

@ 가중평점(Weighted Rating)
 (v/(v+m))*R + (m/(v+m))*C

 - v: 영화별 평점을 투표한 횟수(vote count) > 투표횟수가 많은 영화에 가중치 부여
 - m: 평점을 부여하기 위한 최소 투표 횟수 > 여기서는 투표수 상쉬 60%
 - R: 개별 영화에 대한 평균 평점 (vote average)
 - C: 전체 영화에 대한 평균 평점
 -> C, m은 고정값
 -> v, R은 영화마다 변동값

In [38]:
# 상위 60%에 해당하는 vote count를 최소 투표 횟수인 m으로 지정
C = movies['vote_average'].mean()
m = movies['vote_count'].quantile(0.6)
C, m

(6.092171559442016, 370.1999999999998)

In [40]:
# 가중 평점을 계산하는 함수
def weighted_vote_averate(df:pd.DataFrame):
    v = df['vote_count']
    R = df['vote_average']
    
    return (v/(v+m))*R + (m/(m+v))*C

movies['weighted_vote'] = movies.apply(weighted_vote_averate, axis=1)
movies[['title', 'vote_average', 'genres', 'vote_count', 'weighted_vote']]

,title,vote_average,genres,vote_count,weighted_vote
0,Avatar,7.2,Action Adventure Fantasy Science Fiction,11800,7.166301
1,Pirates of the Caribbean: At World's End,6.9,Adventure Fantasy Action,4500,6.838594
2,Spectre,6.3,Action Adventure Crime,4466,6.284091
3,The Dark Knight Rises,7.6,Action Crime Drama Thriller,9106,7.541095
4,John Carter,6.1,Action Adventure Science Fiction,2124,6.098838
...,...,...,...,...,...
4798,El Mariachi,6.6,Action Crime Thriller,238,6.290894
4799,Newlyweds,5.9,Comedy Romance,5,6.089611
4800,"Signed, Sealed, Delivered",7.0,Comedy Drama Romance TV Movie,6,6.106650
4801,Shanghai Calling,5.7,,7,6.084894


In [43]:
# 장르 유사성이 높은 영화 20개 선정 후 가중 평점순 10개 선정하는 함수

def find_sim_mv(df:pd.DataFrame, sorted_idx:np.ndarray, title_name:str, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_idx[title_index, :top_n*2].reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]
    

In [45]:
similar_movies = find_sim_mv(movies, genre_sim_sorted_idx, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote', 'genres', 'vote_count']]

,title,vote_average,weighted_vote,genres,vote_count
2731,The Godfather: Part II,8.3,8.079586,Drama Crime,3338
1847,GoodFellas,8.2,7.976937,Drama Crime,3128
3866,City of God,8.1,7.759693,Drama Crime,1814
1663,Once Upon a Time in America,8.2,7.657811,Drama Crime,1069
883,Catch Me If You Can,7.7,7.557097,Drama Crime,3795
281,American Gangster,7.4,7.141396,Drama Crime,1502
4041,This Is England,7.4,6.739664,Drama Crime,363
1149,American Hustle,6.8,6.717525,Drama Crime,2807
1243,Mean Streets,7.2,6.626569,Drama Crime,345
2839,Rounders,6.9,6.530427,Drama Crime,439
